# Using PyTorch to perform time-series forecasting

Let's load a PV forecasting dataset:

In [1]:
from chelo import DatasetRegistry

#Load and prepare the train dataset
features = ['temperature', 'radiation_direct_horizontal', 'radiation_diffuse_horizontal',
                'solar_generation_actual']
train_dataset = DatasetRegistry.get_dataset("OPSDPVDataset", end_date='2018-12-31 00:00:00',
                                            selected_features=features,
                                            use_future_weather=True,
                                            prediction_window = 24,
                                            historical_window = 48)
train_dataset.load_data()


# Do the same for the test split
test_dataset = DatasetRegistry.get_dataset("OPSDPVDataset", start_date='2019-1-1 00:00:00',
                                               selected_features=features,
                                               use_future_weather=True, 
                                               prediction_window = 24,
                                               historical_window = 48)
test_dataset.load_data()   
    

We can then get the loaders:

In [2]:
import torch
train_dataset_torch = train_dataset.to_pytorch()
test_dataset_torch = test_dataset.to_pytorch()
train_loader = torch.utils.data.DataLoader(dataset=train_dataset_torch,)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset_torch)

Then, we can define a simple network architecture:

In [3]:
import torch.nn.functional as F
import torch.nn as nn

class MLPRegression(nn.Module):
    def __init__(self, input_dim, output_dim=24, hidden_dim=64):
        super(MLPRegression, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
data_shape = train_dataset.get_features_shape()
input_dim = data_shape[1]*data_shape[2]

Let's define the training and inference functions:

In [4]:
device = 'cpu'
def train_model(model, train_loader, optimizer, criterion, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            images, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            if len(labels.size()) > 1:
                labels = labels.squeeze(-1)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


import torch
import numpy as np
def get_predictions(model, test_loader):
  
    model.eval()  
    all_targets = []  
    all_predictions = [] 

    with torch.no_grad():  # Disable gradient calculations
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Get model predictions
            all_targets.extend(labels.cpu().tolist())
            all_predictions.extend(outputs.cpu().tolist())
    all_targets = np.array(all_targets).squeeze()
    all_predictions = np.array(all_predictions).squeeze()
    return  all_targets, all_predictions  # Return results


Let's perform inference before training our model:

In [5]:
from sklearn.metrics import mean_squared_error, r2_score
model = MLPRegression(input_dim, output_dim=24)

gt, preds = get_predictions(model, test_loader)
print(gt.shape, preds.shape)
print(f"R2: {r2_score(gt, preds)}")
print(f"RMSE: {mean_squared_error(gt, preds)}")

(1446, 24) (1446, 24)
R2: -0.5738519599899988
RMSE: 506752.44333347265


We can then train our model. Note that we can also use a custom loss to this end:

In [6]:
import torch.optim as optim


def custom_loss(x, y):
    return torch.mean((x-y)**2) + torch.mean(torch.abs(x-y))

optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.1)

train_model(model, train_loader, optimizer, custom_loss, num_epochs=5)

Epoch [1/5], Loss: 46564.6437
Epoch [2/5], Loss: 16911.6601
Epoch [3/5], Loss: 14796.3264
Epoch [4/5], Loss: 14074.8644
Epoch [5/5], Loss: 13685.8613


We can then re-evaluate our model:

In [7]:
gt, preds = get_predictions(model, test_loader)
print(gt.shape, preds.shape)
print(f"R2: {r2_score(gt, preds)}")
print(f"RMSE: {mean_squared_error(gt, preds)}")

(1446, 24) (1446, 24)
R2: 0.8759161607135821
RMSE: 41157.43177438994


We can similarly use a recurrent model:

In [8]:
class RecurrentRegression(nn.Module):
    def __init__(self, input_dim, output_dim=24, hidden_dim=64, num_layers=2):
        super(RecurrentRegression, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.transpose(2, 1)
        batch_size = x.size(0)

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device) 
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device) 

        out, _ = self.lstm(x, (h0, c0))  
        out = out[:, -1, :]  
        out = self.fc(out)
        
        return out

In [9]:
model = RecurrentRegression(data_shape[1])

# Train the model:
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.1)

train_model(model, train_loader, optimizer, custom_loss, num_epochs=5)

# Evaluate the model
gt, preds = get_predictions(model, test_loader)
print(gt.shape, preds.shape)
print(f"R2: {r2_score(gt, preds)}")
print(f"RMSE: {mean_squared_error(gt, preds)}")


Epoch [1/5], Loss: 456502.6960
Epoch [2/5], Loss: 443975.9441
Epoch [3/5], Loss: 433027.0965
Epoch [4/5], Loss: 423279.9442
Epoch [5/5], Loss: 414596.2825
(1446, 24) (1446, 24)
R2: -0.36282257097712717
RMSE: 440125.3380877734


What about adding a residual branch to combine both models?

In [10]:
class RecurrentRegression(nn.Module):
    def __init__(self, input_dim, flatten_dim, output_dim=24, hidden_dim=64, num_layers=2):
        super(RecurrentRegression, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.fc1 = nn.Linear(flatten_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.transpose(2, 1)
        batch_size = x.size(0)

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device) 
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device) 

        out, _ = self.lstm(x, (h0, c0))  
        out = out[:, -1, :]  
        out = self.fc(out)
        
        # Residual branch
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        out = out + x
        
        return out
    
    

In [11]:
model = RecurrentRegression(data_shape[1], data_shape[1]*data_shape[2])

# Train the model:
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.1)

train_model(model, train_loader, optimizer, custom_loss, num_epochs=5)

# Evaluate the model
gt, preds = get_predictions(model, test_loader)
print(gt.shape, preds.shape)
print(f"R2: {r2_score(gt, preds)}")
print(f"RMSE: {mean_squared_error(gt, preds)}")

Epoch [1/5], Loss: 44696.5734
Epoch [2/5], Loss: 17062.8541
Epoch [3/5], Loss: 14771.4083
Epoch [4/5], Loss: 14000.6203
Epoch [5/5], Loss: 13594.2798
(1446, 24) (1446, 24)
R2: 0.8843016889758161
RMSE: 38295.84600041278
